In [1]:
import hmftpy as hmf
import numpy as np
import matplotlib.pyplot as plt
import hmftpy as hmf
from hmftpy.plaquettes.square import plaq4
from hmftpy.plaquettes.triangular import plaq12
from quspin.basis import spin_basis_1d
from tqdm import tqdm

First off, let's just diagonalize the AF XY model on a square lattice.

In [2]:
basis = spin_basis_1d(4, pauli=0) # pauli=0 fixes the operators as spin-1/2, rather than pauli matrices
J1 = -1
interactions = {'nearest': {'xx': -J1, 'yy': -J1}}
Hi = hmf.operators.inner_hamiltonian(plaq4, interactions, basis)
e, v = Hi.eigh()
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq4, interactions, basis)
e, v = Hp.eigh()
ep = e[0]
e_hmft, v, mf, cvg = hmf.do_hmft(plaq4, interactions, basis)
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))
print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

ED energy with OBC: -2.828427124746183
ED energy with PBC: -5.656854249492366
HMFT energy: -4.109054536369158
HMFT converged? True


# Turning off bonds

Now, let's add some disorder. How about we turn off bonds randomly. In this package, disorder can be added by passing lists (for local interactions) and matrix (for 2-spin interactions) of numbers in the same dictionary structure used for interactions. These lists/matrices are then used to weight interactions. For instance, if we pass 

    {'nearest': {'xx': r, 'yy': s}}
    
where `r` and `s` are `L` by `L` matrices to a model with interactions

    {'nearest': {'xx': J1, 'yy': J1}},
    
we create the Hamiltonian
\begin{equation}
H = J_1\sum_{\langle i, j \rangle}\left(r_{ij}\sigma_i^x \sigma_j^x + s_{ij}\sigma_i^y \sigma_j^y
\right).
\end{equation}
For the Hamiltonian to be Hermitian, these matrices should be Hermitian as well (or symmetric if they are real).

For our case of turning off bonds (with a roughly 50% chance), we can construct a symmetric matrix of 1s and 0s of size L by L by first constructing a matrix of random floats in this distribution, adding its conjugate, dividing by two, and then rounding to either 0 or 1. We will use the same matrix for both xx and yy terms.

Naively, I expect this to give something around half the initial energy, since I'm removing half the terms from the Hamiltonian. To see if that's the case, I'll average over 20 attempts. Also, to check my matrices are ok, I'll look at the average of the sum of all entries of the matrices. It should be close to 8.

In [12]:
energies = np.zeros(20)
rs = []
rs_sum = np.zeros(20)
for i in tqdm(range(20)):
    r = np.random.rand(4,4)
    r = np.round((r + r.T)/2) # There's some trouble with sparse matrices if I set interactions to exactly zero
    rs += [r]
    rs_sum[i] = np.sum(r)
    disorder = {'nearest': {'xx': r, 'yy': r}} # could make different matrices for the different interactions
    try:
        Hi = hmf.operators.inner_hamiltonian(plaq4, interactions, basis, disorder=disorder)
        e, v = Hi.eigh()
    except:
        print('Null operator')
        print('Noise coefficients for all nn bonds:')
        print(r[0,1])
        print(r[0,2])
        print(r[1,3])
        print(r[2,3])
    energies[i] = e[0]
print('')
print('Average of the sum of entries in the matrices')
print(np.mean(rs_sum))
print('First matrix')
print(rs[0])
print('Average energy (OBC)')
print(np.mean(energies))
print('Compared to half of no-disorder energy')
print(ei/2)

100%|██████████| 20/20 [00:00<00:00, 106.94it/s]


Average of the sum of entries in the matrices
9.0
First matrix
[[0. 1. 1. 1.]
 [1. 1. 0. 1.]
 [1. 0. 1. 1.]
 [1. 1. 1. 0.]]
Average energy (OBC)
-1.8575451317988008
Compared to half of no-disorder energy
-1.4142135623730916


Now, let's try the same thing with HMFT. I'll reuse each matrix so that won't change anything.

In [11]:
energies_hmft = np.zeros(20)
e_hmft,v, mf,cvg = hmf.do_hmft(plaq4, interactions, basis)
cvgs = [False for i in range(20)]
for i, r in enumerate(rs):
    disorder = {'nearest': {'xx': r, 'yy': r}}
    energies_hmft[i], v, mf, cvgs[i] = hmf.do_hmft(plaq4, interactions, basis, disorder=disorder)
print('Average energy (HMFT)')
print(np.mean(energies_hmft))
print('Compared to half of no-disorder energy')
print(e_hmft/2)

Average energy (HMFT)
-2.4276927684078293
Compared to half of no-disorder energy
-2.0545272681845868


# Turning off sites

To remove the nth site from the Hamiltonian, we use the same setup as above, but with the matrix now set to have all entries equal to one except for the nth row and column. In principle, it would be more computationally efficient to rewrite the Hamiltonian and work in a basis where that site doesn't exist at all, but for small clusters the cost is small.

Let's calculate the energy with each site removed (one at a time). In our case (4 site cluster), the sites all have the same number and types of neighbors, so this should give the same energy at each step.

In [13]:
for i in range(4):
    r = np.ones((4,4))
    r[i,:] = 0
    r[:,i] = 0
    disorder = {'nearest': {'xx': r, 'yy': r}}
    energy, v, mf, cvgs = hmf.do_hmft(plaq4, interactions, basis, disorder=disorder)
    print('Energy with the {}th site removed:'.format(i))
    print(energy)

Energy with the 0th site removed:
-2.084565325267061
Energy with the 1th site removed:
-2.084565325267061
Energy with the 2th site removed:
-2.084565325267069
Energy with the 3th site removed:
-2.084565325267056
